In [ ]:
""" 
mem2 
trying to do memory version, with dots fading out over blcoks.
1-line, 2-line, 3-line.
(and some of mem1 at start)
(and one day of mem3,w hcih is last day of entire expt)
"""


In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/data1/code/python/drawmonkey


In [2]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.modelexpt import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

NOTE: need to not overwrite strokes_all_task, because then the orders saved will stop being accurate. Modify


## PLAN
1. Compare tasks (mem) to msot simialr tasks (trace).
- mem are blocks 7, 14, 21 (mem2)
- trace are preceding blocks (take the very first, as later ones arleady sort of faded)
- mistakes due to orientation? location? both?
- write code to find most similar tasks (since these were all random, not fixed).

2. Other behavioral features different for mem?
- reactin time?
- strategy?

3. Learning across days? mem2, control for block number, see improvement over days?






## 3. Learning across days

In [4]:
valstoplot = ["hausdorff_positive", "modelscore", "frac_touched"]
SEPARATE_BY_BLOCK = True # whether should make seperate tiemcourse plotes fore ach block num.
expt = "mem123"


In [5]:
# 1) Load data across days
from pythonlib.tools.datetools import getDateList
from analysis.line2 import PROBEDATfromFD
import seaborn as sns
import pandas as pd
from analysis.line2 import probedat2strokefeats, getStandardizedTime
from analysis.modelexpt import *
from analysis.probedatTaskmodel import *

for animal in ["Red", "Pancho"]:
    FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)
    PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)
    Probedat = ProbedatTaskmodel(PROBEDAT, MD)

    # saving dir
    SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{MD['sdate']}_to_{MD['edate']}/taskmodel"
    import os
    os.makedirs(SAVEDIR, exist_ok=True)
    print(f"saving at {SAVEDIR}")

    if MD["strokmodel_kind"] is not None:
        # ***************************************** COMPUTE MODEL SCORE OFFLIEN
        PROBEDAT, fig = probedatOfflineScore(PROBEDAT, filtdict={"random_task":[False]}, 
                                       ploton=True)
        fig.savefig(f"{SAVEDIR}/offlineScorePairplot.pdf")


    # ********************************************************
    # **************************** extract dataframe
    DF = Probedat.pandas()

    # ********************************************************
    # *********************************** PLOTS
    # === GOOD PLOT, summarize task schedules.
    fig = sns.catplot(x="date", y="task_stagecategory", col = "random_task", row="taskgroup", hue="taskgroup", sharex=True,sharey=False,
                data=DF, height=10, aspect=2, orient="v")
    fig.savefig(f"{SAVEDIR}/overview1.pdf")

    # ==== GOOD PLOT (scatter and means)
    for y in valstoplot:
        ALPHA=0.4
        if y in ["modelcomp", "modelcompflipped"]:
            YLIM=(-0.7, 0.7)
        elif y=="modelcomp_offline":
            YLIM=(-1,1)
        elif y=="hausdorff_positive":
            YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
        elif y=="nstrokesactual":
            YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
            YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
            ALPHA=0.15
        else:
            YLIM = None

        # === 1) All Timecourses
        g = sns.FacetGrid(DF, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                          sharex=True, sharey=True, ylim=YLIM)
        g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
        g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                      "alpha":ALPHA,
                                                              "s":40, 
                                                              "palette":{1:"r", 2:"b"}})
        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
        g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1.pdf")

        # == one plot for each block
        blocks = set(DF["block"])
        if SEPARATE_BY_BLOCK:
            for b in blocks:
                DFblock = DF[DF["block"]==b]
                g = sns.FacetGrid(DFblock, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                          sharex=True, sharey=True, ylim=YLIM)
                g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                              "alpha":ALPHA,
                                                                      "s":40, 
                                                                      "palette":{1:"r", 2:"b"}})
                g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1-block{b}.pdf")

            plt.close("all")

        # === summary (epoch1 vs. epoch2)
        DFsummary = DF[DF["keepforsummary"]==True]
        if len(DFsummary)>0:
            g = sns.FacetGrid(DFsummary, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                              sharex=False, sharey=True, ylim=YLIM)
            g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                  "s":4, 
                                                                  "palette":{1:"r", 2:"b"}})
            g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
            g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary.pdf")

            if SEPARATE_BY_BLOCK:
                for b in blocks:
                    DFsummaryblock = DFsummary[DFsummary["block"]==b]
                    g = sns.FacetGrid(DFsummaryblock, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                      sharex=False, sharey=True, ylim=YLIM)
                    g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                          "s":4, 
                                                                          "palette":{1:"r", 2:"b"}})
                    g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                    g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                    g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary-block{b}.pdf")

                plt.close("all")


    # ***************************************
    # ***************************************
    # [one plot for each unqiue task]
    # === FIRST: THROW OUT RANDOM TASKS
    DF = DF[DF["random_task"]==False]

    if len(DF)>0:
        # ===== get dict of {taskgroup:[list of tasks]}
        tasklist = set(DF["task_stagecategory"])
        taskgrouplist = set(DF["taskgroup"])

        taskgroupdict = {k:[] for k in set([P["taskgroup"] for P in PROBEDAT])}
        for P in PROBEDAT:
            taskgroupdict[P["taskgroup"]].append(P["task_stagecategory"])

        for k, v in taskgroupdict.items():
            taskgroupdict[k] = sorted(set(v))


        # === one figure per taskgroup.
        for taskgroup, tasklist in taskgroupdict.items():
            for task in tasklist:
                DFthis = DF[(DF["taskgroup"]==taskgroup) & (DF["task_stagecategory"]==task)]
                if len(DFthis)>0:
                    for y in valstoplot:
                        ALPHA=0.4
                        if y in ["modelcomp", "modelcompflipped"]:
                            YLIM=(-0.7, 0.7)
                        elif y=="modelcomp_offline":
                            YLIM=(-1,1)
                        elif y=="hausdorff_positive":
                            YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
                        elif y=="nstrokesactual":
                            YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
                            YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
                            ALPHA=0.15

                        g = sns.FacetGrid(DFthis, col="unique_task_name", col_wrap=5, height=4, aspect=2, 
                                          sharex=True, sharey=True, ylim=YLIM)
                        g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                        g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                                      "alpha":ALPHA,
                                                                              "s":40, 
                                                                              "palette":{1:"r", 2:"b"}})
                        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                        g.savefig(f"{SAVEDIR}/timecourses-{taskgroup}-{task}-{y}-1.pdf")

                        # === summary
                        DFsummary = DFthis[DFthis["keepforsummary"]==True]
                        if len(DFsummary)>0:
                            g = sns.FacetGrid(DFsummary, col="unique_task_name", col_wrap=5, height=4, aspect=1.2, 
                              sharey=True, ylim=YLIM)
                            g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                                  "s":4, 
                                                                                  "palette":{1:"r", 2:"b"}})
                            g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                            g.savefig(f"{SAVEDIR}/timecourses-{taskgroup}-{task}-{y}-summary.pdf")


        # === ONE LINE PER UNIQUE TASK
        DFsummary = DF[DF["keepforsummary"]==True]

        # aggregate over unique tasks
        values = valstoplot
        from pythonlib.tools.pandastools import aggregGeneral
        DFsummary = aggregGeneral(DFsummary, ["unique_task_name", "epoch", "taskgroup"], values, nonnumercols=["task_stagecategory"], aggmethod=["median"])

        # == only keep cases that have data for both epochs.
        values_to_check = [1,2]
        colname = "epoch"
        group = "unique_task_name"
        from pythonlib.tools.pandastools import filterGroupsSoNoGapsInData
        DFsummary = filterGroupsSoNoGapsInData(DFsummary, group, colname, values_to_check)

        # === PLOT
        for y in values:
            from pythonlib.tools.snstools import relplotOverlaid
            fig = relplotOverlaid(DFsummary, "unique_task_name", "k",
                           data=DFsummary, x="epoch", y=y, col="taskgroup", row="task_stagecategory", 
                    hue="unique_task_name", kind="line")
            fig.savefig(f"{SAVEDIR}/summarypaired-{y}-1.pdf")

            fig = sns.catplot(data=DFsummary, x="epoch", y=y, col="taskgroup", row="task_stagecategory",  kind="point")
            fig.savefig(f"{SAVEDIR}/summarypaired-{y}-2.pdf")

This is metadat:
{'sdate': 201213, 'edate': 201220, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'201213': 1, '201214': 1, '201215': 1, '201216': 1, '201217': 1, '201218': 1, '201219': 1, '201220': 1}, 'dates_for_summary': [], 'matchedstrokes': None, 'task_train_test': {'probe1_liketrain': 'train', 'probe1_nostrokeconstraint': 'train', 'probe2_liketrain': 'train', 'probe2_nostrokeconstraint': 'train', 'probe3_hdpos': 'test', 'probe1': 'train', 'probe2': 'train', 'probe3': 'test', 'probe4': 'test', 'train': 'train'}, 'expt': 'mem123', 'exptnames': ['mem1', 'mem2', 'mem3']}

Getting these dates:
['201213', '201214', '201215', '201216', '201217', '201218', '201219', '201220']
Red 201213 mem1 0
- No h5 file for Red, 201213, mem1, 0 - returning None!
Red 201213 mem1 1
-- loaded presaved data: /data2/animals/Red/201213/201213_151702_mem1_Red_1.pkl
got 200 total trials
appending fd for sess 1
Red 201213 mem1 2
-- loaded presaved data: /data2/animals/Red/201213/201213

-- loaded presaved data: /data2/animals/Red/201216/201216_151520_mem2_Red_1.pkl
got 795 total trials
appending fd for sess 1
Red 201216 mem2 2
- No h5 file for Red, 201216, mem2, 2 - returning None!
Red 201216 mem2 3
- No h5 file for Red, 201216, mem2, 3 - returning None!
Red 201216 mem2 4
- No h5 file for Red, 201216, mem2, 4 - returning None!
Red 201216 mem2 5
- No h5 file for Red, 201216, mem2, 5 - returning None!
Red 201216 mem2 6
- No h5 file for Red, 201216, mem2, 6 - returning None!
Red 201216 mem2 7
- No h5 file for Red, 201216, mem2, 7 - returning None!
Red 201216 mem2 8
- No h5 file for Red, 201216, mem2, 8 - returning None!
Red 201216 mem2 9
- No h5 file for Red, 201216, mem2, 9 - returning None!
Red 201216 mem3 0
- No h5 file for Red, 201216, mem3, 0 - returning None!
Red 201216 mem3 1
- No h5 file for Red, 201216, mem3, 1 - returning None!
Red 201216 mem3 2
- No h5 file for Red, 201216, mem3, 2 - returning None!
Red 201216 mem3 3
- No h5 file for Red, 201216, mem3, 3 - ret

-- loaded presaved data: /data2/animals/Red/201220/201220_142124_mem3_Red_1.pkl
got 846 total trials
appending fd for sess 1
Red 201220 mem3 2
- No h5 file for Red, 201220, mem3, 2 - returning None!
Red 201220 mem3 3
- No h5 file for Red, 201220, mem3, 3 - returning None!
Red 201220 mem3 4
- No h5 file for Red, 201220, mem3, 4 - returning None!
Red 201220 mem3 5
- No h5 file for Red, 201220, mem3, 5 - returning None!
Red 201220 mem3 6
- No h5 file for Red, 201220, mem3, 6 - returning None!
Red 201220 mem3 7
- No h5 file for Red, 201220, mem3, 7 - returning None!
Red 201220 mem3 8
- No h5 file for Red, 201220, mem3, 8 - returning None!
Red 201220 mem3 9
- No h5 file for Red, 201220, mem3, 9 - returning None!
===== SUMMARY
--
Red-201213-1: ntrials: 200

--
Red-201213-2: ntrials: 419

--
Red-201214-1: ntrials: 167

--
Red-201214-2: ntrials: 33

--
Red-201214-3: ntrials: 45

--
Red-201214-4: ntrials: 11

--
Red-201214-5: ntrials: 124

--
Red-201214-6: ntrials: 15

--
Red-201214-7: ntrials:

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interfa

This is metadat:
{'sdate': 201213, 'edate': 201220, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'201213': 1, '201214': 1, '201215': 1, '201216': 1, '201217': 1, '201218': 1, '201219': 1, '201220': 1}, 'dates_for_summary': [], 'matchedstrokes': None, 'task_train_test': {'probe1_liketrain': 'train', 'probe1_nostrokeconstraint': 'train', 'probe2_liketrain': 'train', 'probe2_nostrokeconstraint': 'train', 'probe3_hdpos': 'test', 'probe1': 'train', 'probe2': 'train', 'probe3': 'test', 'probe4': 'test', 'train': 'train'}, 'expt': 'mem123', 'exptnames': ['mem1', 'mem2', 'mem3']}

Getting these dates:
['201213', '201214', '201215', '201216', '201217', '201218', '201219', '201220']
Pancho 201213 mem1 0
- No h5 file for Pancho, 201213, mem1, 0 - returning None!
Pancho 201213 mem1 1
-- loaded presaved data: /data2/animals/Pancho/201213/201213_160114_mem1_Pancho_1.pkl
got 560 total trials
appending fd for sess 1
Pancho 201213 mem1 2
- No h5 file for Pancho, 201213, mem1, 

-- loaded presaved data: /data2/animals/Pancho/201216/201216_165347_mem2_Pancho_2.pkl
got 463 total trials
appending fd for sess 2
Pancho 201216 mem2 3
- No h5 file for Pancho, 201216, mem2, 3 - returning None!
Pancho 201216 mem2 4
- No h5 file for Pancho, 201216, mem2, 4 - returning None!
Pancho 201216 mem2 5
- No h5 file for Pancho, 201216, mem2, 5 - returning None!
Pancho 201216 mem2 6
- No h5 file for Pancho, 201216, mem2, 6 - returning None!
Pancho 201216 mem2 7
- No h5 file for Pancho, 201216, mem2, 7 - returning None!
Pancho 201216 mem2 8
- No h5 file for Pancho, 201216, mem2, 8 - returning None!
Pancho 201216 mem2 9
- No h5 file for Pancho, 201216, mem2, 9 - returning None!
Pancho 201216 mem3 0
- No h5 file for Pancho, 201216, mem3, 0 - returning None!
Pancho 201216 mem3 1
- No h5 file for Pancho, 201216, mem3, 1 - returning None!
Pancho 201216 mem3 2
- No h5 file for Pancho, 201216, mem3, 2 - returning None!
Pancho 201216 mem3 3
- No h5 file for Pancho, 201216, mem3, 3 - retur

-- loaded presaved data: /data2/animals/Pancho/201220/201220_143442_mem3_Pancho_1.pkl
got 821 total trials
appending fd for sess 1
Pancho 201220 mem3 2
- No h5 file for Pancho, 201220, mem3, 2 - returning None!
Pancho 201220 mem3 3
- No h5 file for Pancho, 201220, mem3, 3 - returning None!
Pancho 201220 mem3 4
- No h5 file for Pancho, 201220, mem3, 4 - returning None!
Pancho 201220 mem3 5
- No h5 file for Pancho, 201220, mem3, 5 - returning None!
Pancho 201220 mem3 6
- No h5 file for Pancho, 201220, mem3, 6 - returning None!
Pancho 201220 mem3 7
- No h5 file for Pancho, 201220, mem3, 7 - returning None!
Pancho 201220 mem3 8
- No h5 file for Pancho, 201220, mem3, 8 - returning None!
Pancho 201220 mem3 9
- No h5 file for Pancho, 201220, mem3, 9 - returning None!
===== SUMMARY
--
Pancho-201213-1: ntrials: 561

--
Pancho-201214-1: ntrials: 124

--
Pancho-201214-2: ntrials: 64

--
Pancho-201214-3: ntrials: 124

--
Pancho-201214-4: ntrials: 65

--
Pancho-201214-5: ntrials: 104

--
Pancho-201

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interfa

## More plots, Plot task behaviors (below not cleaned up, but works)
### LOTS OF PLOTS (ALL TRIALS, EXAMPLE TRIALS, ETC)

In [ ]:
expt = "mem123"
thingstoplot = ["alltrials", "egtrials"]

# expt = "lines5"
# # max_strokenum = 2 # for single stroke plots
# thingstoplot = ["alltrials", "egtrials"]
# # thingstoplot = ["egtrials"]

In [ ]:
def plotalltrials(PROBEDATthis, fdlist, stage, datelist, tasklist):
    tasklist = sorted(set([P["unique_task_name"] for P in PROBEDATthis if P["task_stagecategory"]==stage]))
    if len(tasklist)>100:
        assert False, "why so many tasks for this stage?"

    # -- Plot 2d grid, separated by days
    CAT1 = ["date", datelist]
    CAT2 = ["unique_task_name", tasklist]
    fdlist = None

    # == 1) All behavior trials, all strokes overlaid
    plotargs = {"zoom":True, "plotver":"order", "markersize":4, "alpha":0.25}
    # fdlist = [P["filedata"][0]() for P in PROBEDATthis]
    # fdlist = [FD[P["ii"]]["fd"] for P in PROBEDATthis]

    fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="beh", plotargs=plotargs);
    fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_beh_{stage}.pdf")
    fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="task", plotargs=plotargs);
    fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_task_{stage}.pdf")

    # == 2) Split by stroke number (one plot for each stroke)
    if False:
        plotargs = {"zoom":True, "plotver":"order", "markersize":3, "alpha":0.2}
        strokenums_to_plot_alone=list(range(max_strokenum+1))
        overlay_stroke_mean=False

        fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="behtask", 
                              plotargs=plotargs, strokenums_to_plot_alone=strokenums_to_plot_alone, 
                             overlay_stroke_mean = overlay_stroke_mean);
        fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_eachstroke_beh_{stage}.pdf")


    # == 3) All strokes (faint) and overlay average
    if False:
        plotargs = {"zoom":True, "plotver":"order", "markersize":2, "alpha":0.15}

        overlay_stroke_mean=True
        fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="beh", 
                              plotargs=plotargs, overlay_stroke_mean=overlay_stroke_mean);
        fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_strokemeans_{stage}.pdf")


In [ ]:
debug=False

In [ ]:
# Plot same task across days

# 1) Load data across days
from pythonlib.tools.datetools import getDateList
# expt = "lines2"
for animal in ["Pancho", "Red"]:
    
    FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)
    PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)

    # saving dir
    sdate = MD["sdate"]
    edate = MD["edate"]
    SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{sdate}_to_{edate}"
    import os
    os.makedirs(SAVEDIR, exist_ok=True)
    print(f"saving at {SAVEDIR}")

    # get all fixed tasks of a particular kind
    kindlist = set([P["kind"] for P in PROBEDAT if P["random_task"]==False])
    task_per_kind = {}
    for kind in kindlist:
        tasklist = set([P["unique_task_name"] for P in PROBEDAT if P["kind"]==kind])
        task_per_kind[kind]=sorted(tasklist)

    print("tasks per kind found")
    for k, v in task_per_kind.items():
        print("----")
        print(f"=={k}")
        [print(vv) for vv in v]


    ## PLOT - all trials, 2d grid sorted by date and task category
    # only keep data for fixed tasks.
    PROBEDATthis = [P for P in PROBEDAT if P["random_task"]==False]
    datelist = sorted(set([P["date"] for P in PROBEDATthis]))
    fdlist = None
    
    # for each stage, make a 2d grid plot (date x task)
    if "alltrials" in thingstoplot:
        # all categories that have fixed tasks
        stagelist = set([P["task_stagecategory"] for P in PROBEDATthis if P["random_task"]==False])
        for stage in stagelist:
            plotalltrials(PROBEDATthis, fdlist, stage, datelist, tasklist)

    # ==== PLOT ALL TRIALS
    if "egtrials" in thingstoplot:
        if debug:
            # making plots for lab meeting..
            tasklist = set([P["unique_task_name"] for P in PROBEDATthis if P["random_task"]==False
               and (P["task_stagecategory"] in ["LplusL", "2linePlusL", "3linePlusL"] or "linePlusLv2_51" in P["unique_task_name"])])
        else:
            tasklist = set([P["unique_task_name"] for P in PROBEDATthis if P["random_task"]==False])
        NMAX = 20 # trials to plot, starting from 1st trial int he day
        for task in tasklist:

            PD = [P for P in PROBEDATthis if P["random_task"]==False and P["unique_task_name"]==task]

            for reverse in [False, True]:
                # -- task presentation num as column
                for P in PD:
                    P["idx_today_uniquetask"] = None
                PD, countlist = probeDatIndexWithinDay(PD, task, reverse_order=reverse);

                # -- how many examples to plot?
                ntoplot = min((max(countlist), NMAX))

                # -- Plot 2d grid, separated by days
                CAT1 = ["date", datelist]
                CAT2 = ["idx_today_uniquetask", range(ntoplot)]
                fdlist = None

                # == 1) All behavior trials, all strokes overlaid
                plotargs = {"zoom":True, "plotver":"order", "markersize":8, "alpha":0.7}
                plot_task_last_col = True
                ver = "beh"

                fig = plotTrial2dGrid(PD, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver=ver, 
                                      plotargs=plotargs, plot_task_last_col=plot_task_last_col);
                if reverse:
                    fig.savefig(f"{SAVEDIR}/egtrials_datebyexample_revchronorder_{task}.pdf")
                else:
                    fig.savefig(f"{SAVEDIR}/egtrials_datebyexample_chronorder_{task}.pdf")

        plt.close("all")

## Plot all tasks (no behavior)


IN PROGRESS - not sure if this is good

In [ ]:
# ************** PLOT ALL FIXED TASKS (collect across days)
for TRAINTEST in ["train", "test"]:

    # then plot all
    tasknames = sorted(set([P["unique_task_name"] for P in PROBEDAT
                      if P["random_task"]==False
                            and task_train_test[P["kind"]]==TRAINTEST]))

    # get one index per taskname
    fdlist = []
    triallist =[]
    titlelist = []
    for task in tasknames:
        fdlist.append([P["filedata"] for P in PROBEDAT
                        if P["unique_task_name"]==task][0])
        triallist.append([P["trial"] for P in PROBEDAT
                        if P["unique_task_name"]==task][0])
        titlelist.append(task)
    print(titlelist)

    fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=None, 
                               empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                        plotargs={"plot_task_stimulus":True, 
                                "plot_drawing_behavior":False, "nakedplot":True,
                               })
    fig.savefig(f"{SAVEDIR}/alltasks_fixed_alldays_{TRAINTEST}.pdf")

    
    
# *** FOR EACH DAY, PLOT SUBSET OF RANDOM TASKS.
TRAINTEST = "train"
for date in DATELIST:
    print(f"======= {date}")
    traintaskcats = sorted(set([P["task_stagecategory"] for P in PROBEDAT 
                                if P["date"]==date
                               and P["random_task"]==True
                               and task_train_test[P["kind"]]==TRAINTEST]))
    for cat in traintaskcats:
        print(f"--- {cat}")
        
        # then just plot example tasks
        fdlist = [P["filedata"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]
        triallist = [P["trial"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]
        titlelist = [P["unique_task_name"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]

        Nrand = 60
        fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=Nrand, 
                                   empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                            plotargs={"plot_task_stimulus":True, 
                                    "plot_drawing_behavior":False, "nakedplot":True,
                                   })
        fig.savefig(f"{SAVEDIR}/alltasks-random-{date}-{cat}-{TRAINTEST}.pdf")
        plt.close()


## BELOW - OLD SCRATCH

## SCRATCH